In [3]:
#CLM
!pip install transformers datasets

In [4]:
#load data
from datasets import load_dataset

eli5 = load_dataset("eli5", split="train_asks[:5000]")

Downloading: 100%|██████████| 3.50k/3.50k [00:00<00:00, 3.49MB/s]
Downloading: 100%|██████████| 576M/576M [03:49<00:00, 2.51MB/s] 
Downloading: 100%|██████████| 21.1M/21.1M [00:06<00:00, 3.13MB/s]
Downloading: 100%|██████████| 53.0M/53.0M [00:23<00:00, 2.22MB/s]
Downloading: 100%|██████████| 286M/286M [01:46<00:00, 2.69MB/s] 
Downloading: 100%|██████████| 9.65M/9.65M [00:04<00:00, 2.34MB/s]
Downloading: 100%|██████████| 17.7M/17.7M [00:07<00:00, 2.51MB/s]
Downloading: 100%|██████████| 330M/330M [01:31<00:00, 3.61MB/s] 
Downloading: 100%|██████████| 18.7M/18.7M [00:05<00:00, 3.29MB/s]
Downloading: 100%|██████████| 36.2M/36.2M [00:09<00:00, 3.72MB/s]


In [7]:
#take a look
eli5=["train"][0]

In [8]:
#preprocess
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

In [12]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

In [13]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

AttributeError: 'str' object has no attribute 'map'

In [14]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [15]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

NameError: name 'tokenized_eli5' is not defined

In [16]:
#Train
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: c99e6aec-d73a-4700-a039-35a888f97d18)')' thrown while requesting HEAD https://huggingface.co/distilgpt2/resolve/main/generation_config.json


In [1]:
training_args = TrainingArguments(
    #output_dir="my_awesome_eli5_clm-model",
    output_dir=f'{model}-finetuned-model',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

SyntaxError: invalid syntax (Temp/ipykernel_9456/2246215539.py, line 4)

In [ ]:
# get perplexity
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.push_to_hub()